In [48]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [49]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

init = tf.global_variables_initializer()


Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [50]:
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          keep_prob: 0.7})
        total_cost += cost_val

    print('에폭:', '%04d' % (epoch + 1),
          '평균 cost =', '{:.3f}'.format(total_cost / total_batch))

print('모델 완성!')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy,
                        feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
                                   Y: mnist.test.labels,
                                   keep_prob: 1}))

Epoch: 0001 Avg. cost = 0.342
Epoch: 0002 Avg. cost = 0.109
Epoch: 0003 Avg. cost = 0.076
Epoch: 0004 Avg. cost = 0.061
Epoch: 0005 Avg. cost = 0.051
Epoch: 0006 Avg. cost = 0.041
Epoch: 0007 Avg. cost = 0.036
Epoch: 0008 Avg. cost = 0.030
Epoch: 0009 Avg. cost = 0.027
Epoch: 0010 Avg. cost = 0.024
Epoch: 0011 Avg. cost = 0.020
Epoch: 0012 Avg. cost = 0.019
Epoch: 0013 Avg. cost = 0.018
Epoch: 0014 Avg. cost = 0.017
Epoch: 0015 Avg. cost = 0.014
모델 완성!
정확도: 0.9916


In [51]:
#tf.layers 서브 패키지 활용한 cnn 모델링

In [60]:
x=tf.placeholder(tf.float32, [None, 28, 28, 1])
y=tf.placeholder(tf.float32, [None, 10])

In [61]:
tf.reset_default_graph()
L1=tf.layers.conv2d(x, 32, [3,3], activation=tf.nn.relu)
L1=tf.layers.max_pooling2d(L1,[2,2],[2,2])
L1=tf.layers.dropout(L1,0.7)

L2=tf.layers.conv2d(L1, 64, [3,3], activation=tf.nn.relu)
L2=tf.layers.max_pooling2d(L2,[2,2],[2,2])
L2=tf.layers.dropout(L2,0.7)

L3=tf.contrib.layers.flatten(L2)
L3=tf.layers.dense(L3,256, activation=tf.nn.relu)
L3=tf.layers.dropout(L3,0.5)

model=tf.layers.dense(L3,10, activation=None)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2
                    (logits=model, labels=y))
train=tf.train.AdamOptimizer(0.001).minimize(cost)

ValueError: in converted code:
    relative to C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python:

    keras\layers\convolutional.py:196 call *
        outputs = self._convolution_op(inputs, self.kernel)
    ops\nn_ops.py:1079 __call__
        return self.conv_op(inp, filter)
    ops\nn_ops.py:635 __call__
        return self.call(inp, filter)
    ops\nn_ops.py:234 __call__
        name=self.name)
    ops\nn_ops.py:1953 conv2d
        name=name)
    ops\gen_nn_ops.py:1160 conv2d
        data_format=data_format, dilations=dilations, name=name)
    framework\op_def_library.py:366 _apply_op_helper
        g = ops._get_graph_from_inputs(_Flatten(keywords.values()))
    framework\ops.py:6135 _get_graph_from_inputs
        _assert_same_graph(original_graph_element, graph_element)
    framework\ops.py:6071 _assert_same_graph
        (item, original_item))

    ValueError: Tensor("conv2d/kernel:0", shape=(3, 3, 1, 32), dtype=float32_ref) must be from the same graph as Tensor("Placeho

In [ ]:
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          keep_prob: 0.7})
        total_cost += cost_val

    print('에폭:', '%04d' % (epoch + 1),
          '평균 cost =', '{:.3f}'.format(total_cost / total_batch))

print('모델 완성!')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy,
                        feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
                                   Y: mnist.test.labels,
                                   keep_prob: 1}))

In [62]:
#keras에서 mnist cnn모델링

In [63]:
from keras import layers
from keras import models

Using TensorFlow backend.


In [65]:
model=models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', 
                        input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64, (3,3), activation='relu'))

W0819 14:45:46.436188  1084 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0819 14:45:46.436188  1084 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 14:45:46.436188  1084 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0819 14:45:46.451572  1084 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [66]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [67]:
model.add(layers.Flatten())

In [68]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [69]:
model.add(layers.Dense(64, activation='relu'))

In [70]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
Total para

In [71]:
model.add(layers.Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

In [72]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [73]:
(train_images, train_labels), (test_images, test_labels)=
mnist.load_data()

In [76]:
train_images.shape
train_images=train_images.reshape((60000, 28, 28, 1))
train_images=train_images.astype('float32')/255
test_images=test_images.reshape((10000, 28, 28, 1))
test_images=test_images.astype('float32')/255

In [79]:
train_labels=to_categorical(train_labels)
test_labels=to_categorical(test_labels)

In [81]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

W0819 15:05:03.234409  1084 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0819 15:05:03.250061  1084 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [82]:
model.fit(train_images, train_labels, epochs=5, batch_size=64)

W0819 15:05:58.091598  1084 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0819 15:05:58.195022  1084 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/5
60000/60000 [==============================] - 27s 448us/step - loss: 0.1698 - acc: 0.9480
Epoch 2/5
60000/60000 [==============================] - 27s 448us/step - loss: 0.0476 - acc: 0.9852
Epoch 3/5
60000/60000 [==============================] - 26s 440us/step - loss: 0.0320 - acc: 0.99041s - loss: 
Epoch 4/5
60000/60000 [==============================] - 26s 440us/step - loss: 0.0246 - acc: 0.9924
Epoch 5/5
60000/60000 [==============================] - 26s 440us/step - loss: 0.0194 - acc: 0.9941


In [83]:
test_loss, test_acc=model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 1s 129us/step


0.9911